# Analyze the mutations

Read mutations in the new variants:

In [1]:
import re

import pandas as pd

pd.set_option("display.max_colwidth", 150)

new_variants = (
    pd.read_csv("gisaid_seqs/gisaid_hcov-19_2023_08_14_18.csv", sep=";")
    .assign(name=lambda x: x["seqName"].str.split("|").str[0])
    .melt(
        id_vars="name",
        value_vars=["aaSubstitutions", "aaDeletions", "aaInsertions"],
        var_name="mutation_type",
        value_name="mutation",
    )
    .assign(
        mutation=lambda x: x["mutation"].str.split(","),
        seq_type="new variant",
        
    )
    .explode("mutation")
    .query("mutation.notnull()")
)

Read standard lineages (clades), get just those of interest:

In [2]:
lineage_names = ["BA.2", "XBB.1.5"]  # lineages of interest

lineages = (
    pd.read_json("pango-consensus-sequences_summary.json", orient="index")
    .melt(
        id_vars="lineage",
        value_vars=["aaSubstitutions", "aaDeletions"],
        value_name="mutation",
        var_name="mutation_type",
    )
    .explode("mutation")
    .rename(columns={"lineage": "name"})
    .assign(seq_type="lineage")
    .query("mutation != ''")
    .query("name in @lineage_names")
    .reset_index(drop=True)
)

Combine the mutations of the sequences of interest and lineages and break them down by site:

In [3]:
muts = (
    pd.concat([lineages, new_variants])
    .assign(
        gene=lambda x: x["mutation"].str.split(":").str[0],
        mutation=lambda x: x["mutation"].str.split(":", n=1).str[1],
        site=lambda x: x.apply(
            lambda r: (
                int(r["mutation"][1: -1])
                if r["mutation_type"] != "aaInsertions"
                else int(r["mutation"].split(":")[0])
            ),
            axis=1,
        ),
    )
    .drop(columns="mutation_type")
)

Some of the new sequences are missing coverage, get what that is:

In [4]:
new_variant_missing_coverage = (
    pd.read_csv("gisaid_seqs/gisaid_hcov-19_2023_08_14_18.csv", sep=";")
    .assign(
        name=lambda x: x["seqName"].str.split("|").str[0],
        missing=lambda x: x["unknownAaRanges"].str.split(","),
    )
    .explode("missing")
    .assign(
        gene=lambda x: x["missing"].str.split(":").str[0],
        site_range=lambda x: x["missing"].str.split(":", n=1).str[1].str.split("-"),
        site=lambda x: x["site_range"].map(
            lambda r: [int(r[0])] if len(r) == 1 else list(range(int(r[0]), int(r[1]) + 1))
        ),
    )
    .explode("site")
    [["name", "gene", "site"]]
)

Get the consensus mutations in the new variants, keeping track of which ones are missing coverage:

In [5]:
assert set(new_variant_missing_coverage["name"]).issubset(muts["name"])

new_variant_muts = (
    muts
    .query("seq_type == 'new variant'")
    .merge(
        new_variant_missing_coverage,
        how="left",
        on=["gene", "site"],
        suffixes=["", "_missing"],
    )
    .groupby(["name", "mutation", "gene", "site"], as_index=False)
    .aggregate(
        missing_site=pd.NamedAgg("name_missing", "unique"),
        n_missing_site=pd.NamedAgg("name_missing", "nunique"),
    )
    .assign(missing_site=lambda x: x["missing_site"].map(lambda a: "" if pd.isnull(a).all() else ";".join(a)))
    .groupby(["gene", "site", "mutation", "n_missing_site", "missing_site"], as_index=False)
    .aggregate(
        n_with_mutation=pd.NamedAgg("name", "nunique"),
        has_mutation=pd.NamedAgg("name", "unique"),
    )
    .assign(
        n_lacking_mutation=lambda x: new_variants["name"].nunique() - x["n_missing_site"] - x["n_with_mutation"],
        with_mutation=lambda x: x["has_mutation"].map(lambda a: "; ".join(a)),
    )
    .sort_values(["gene", "site"])
    [["gene", "site", "mutation", "n_with_mutation", "n_lacking_mutation", "n_missing_site", "with_mutation"]]
)

new_variant_muts

,gene,site,mutation,n_with_mutation,n_lacking_mutation,n_missing_site,with_mutation
0,E,9,T9I,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
1,M,3,D3H,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
2,M,19,Q19E,2,0,1,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Israel/ICH-741198454/2023
3,M,30,T30A,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
4,M,63,A63T,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
...,...,...,...,...,...,...,...
100,S,796,D796Y,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
101,S,939,S939F,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
102,S,954,Q954H,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023
103,S,969,N969K,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023


Now get new mutations relative to each lineage:

In [6]:
lineage_muts = (
    muts.query("seq_type == 'lineage'")
    .assign(has_mutation=True)
    .pivot_table(
        index=["gene", "site", "mutation"],
        columns="name",
        values="has_mutation",
        fill_value=False,
        aggfunc="any",
    )
    .reset_index()
)
lineage_muts["Wuhan-Hu-1"] = False
lineage_names.append("Wuhan-Hu-1")

muts_from_lineages = (
    new_variant_muts
    .merge(lineage_muts, how="outer", validate="one_to_one", on=["gene", "site", "mutation"])
    .assign(
        site=lambda x: x["site"].astype(int),
        n_with_mutation=lambda x: x["n_with_mutation"].fillna(0).astype(int),
        n_lacking_mutation=lambda x: x["n_lacking_mutation"].fillna(0).astype(int),
        n_missing_site=lambda x: x["n_missing_site"].fillna(0).astype(int),
        with_mutation=lambda x: x["with_mutation"].fillna(""),
    )
    .fillna(False)
)

muts_from_lineages

/tmp/ipykernel_36741/1224619884.py:18: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  .merge(lineage_muts, how="outer", validate="one_to_one", on=["gene", "site", "mutation"])


,gene,site,mutation,n_with_mutation,n_lacking_mutation,n_missing_site,with_mutation,BA.2,XBB.1.5,Wuhan-Hu-1
0,E,9,T9I,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023,True,True,False
1,M,3,D3H,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023,False,False,False
2,M,19,Q19E,2,0,1,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Israel/ICH-741198454/2023,True,True,False
3,M,30,T30A,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023,False,False,False
4,M,63,A63T,3,0,0,hCoV-19/Denmark/DCGC-647646/2023; hCoV-19/Denmark/DCGC-647676/2023; hCoV-19/Israel/ICH-741198454/2023,True,True,False
...,...,...,...,...,...,...,...,...,...,...
119,S,445,V445P,0,0,0,,False,True,False
120,S,484,E484A,0,0,0,,True,True,False
121,S,490,F490S,0,0,0,,False,True,False
122,S,493,Q493R,0,0,0,,True,False,False


Now enumerate each set of mutations relative to each lineage, just for spike:

In [7]:
def reconcile_site_muts(g_df):
    """If site is mutation in both variant and lineage, reconcile."""
    if (len(g_df) == 1) and g_df["n_with_mutation"].all():
        return g_df["mutation"].values[0]
    elif len(g_df) == 1:
        mut = g_df["mutation"].values[0]
        assert re.fullmatch("[A-Z\-]\d+[A-Z\-]", mut), mut
        return mut[-1] + mut[1: -1] + mut[0]
    elif len(g_df.query("n_with_mutation == 0")):
        assert len(g_df) == 2
        for_mut = g_df.query("n_with_mutation > 0")["mutation"].values[0]
        rev_mut = g_df.query("n_with_mutation == 0")["mutation"].values[0]
        assert re.fullmatch("[A-Z\-]\d+[A-Z\-]", for_mut), for_mut
        assert re.fullmatch("[A-Z\-]\d+[A-Z\-]", rev_mut), rev_mut
        return rev_mut[-1] + for_mut[1: ]
    else:
        raise ValueError(g_df)

for lineage in lineage_names:
    
    print(f"\nGetting mutations relative to {lineage=}")
    df = (
        muts_from_lineages
        .query(f"(n_with_mutation > 0) != `{lineage}`")
        .drop(columns=lineage_names)
        .query("gene == 'S'")
    )
    
    df = (
        df
        .groupby(["gene", "site"])
        .apply(reconcile_site_muts).rename("mutation").reset_index()
        .merge(
            df.query("n_with_mutation > 0").drop(columns="mutation"),
            on=["gene", "site"],
            validate="one_to_one",
            how="left",
        )
        .assign(
            n_with_mutation=lambda x: x["n_with_mutation"].fillna(new_variants["name"].nunique()).astype(int),
            n_lacking_mutation=lambda x: x["n_lacking_mutation"].fillna(0).astype(int),
            n_missing_site=lambda x: x["n_missing_site"].fillna(0).astype(int),
            with_mutation=lambda x: x["with_mutation"].fillna("; ".join(new_variants["name"].unique())),
        )
        .sort_values(["gene", "site"])
        .reset_index(drop=True)
    )
    
    outfile = f"spike_aa_muts_relative_to_{lineage}.csv"
    print(f"Writing {len(df)} mutations to {outfile}")
    df.to_csv(outfile, index=False)


Getting mutations relative to lineage='BA.2'
Writing 35 mutations to spike_aa_muts_relative_to_BA.2.csv

Getting mutations relative to lineage='XBB.1.5'
Writing 37 mutations to spike_aa_muts_relative_to_XBB.1.5.csv

Getting mutations relative to lineage='Wuhan-Hu-1'
Writing 61 mutations to spike_aa_muts_relative_to_Wuhan-Hu-1.csv


Also get mutations in XBB.1.5 relative to BA.2:

In [8]:
def ba2_to_xbb15(g_df):
    if len(g_df) == 1 and g_df["XBB.1.5"].all():
        return g_df["mutation"].values[0]
    elif len(g_df) == 1 and g_df["BA.2"].all():
        mut = g_df["mutation"].values[0]
        return mut[-1] + mut[1: -1] + mut[0]
    else:
        assert len(g_df) == 2
        for_mut = g_df.query("`XBB.1.5`")["mutation"].values[0]
        rev_mut = g_df.query("`BA.2`")["mutation"].values[0]
        return rev_mut[-1] + for_mut[1: ]

(
    lineage_muts
    .query("`BA.2` != `XBB.1.5`")
    .groupby(["gene", "site"], as_index=False)
    .apply(ba2_to_xbb15)
    .query("gene == 'S'")
)

,gene,site,None
6,S,83,V83A
7,S,144,Y144-
8,S,146,H146Q
9,S,183,Q183E
10,S,213,G213E
11,S,252,G252V
12,S,339,D339H
13,S,346,R346T
14,S,368,L368I
15,S,445,V445P
